**Imports**

In [1]:
import pyvisa
import tkinter as tk

**PYVISA AND TCP DEVICE CONNECTION**

In [ ]:
# Replace with the correct IP address and port of your probe bench
IP_ADDRESS = "192.168.0.100"  
PORT = 5025  # Example port, replace with the correct one

rm = pyvisa.ResourceManager()
connection_string = f"TCPIP::{IP_ADDRESS}::{PORT}::SOCKET"
probe_bench = rm.open_resource(connection_string)

# Check connection
response = probe_bench.query("*IDN?")
print(f"Connected to: {response}")


**DICTIONARY FOR TRANSLATING COMMAND STRING LITERALS**

In [21]:
# Dictionary for Chuck and Scope commands mapped to ASCII codes
# Dictionary mapping command names to their respective Command IDs
command_dict = {
    # Chuck Commands
    "InitChuck": '33',  
    # Initializes the chuck stage. Moves specified axis to negative end limit, then off end limit. Resets position counter to zero.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is All Axes)

    "MoveChuck": '34',  
    # Moves the chuck stage to the specified X, Y position relative to the wafer home position.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckAlign": '38', 
    # Moves the chuck Z axis to the align height.
    # Parameters: None

    "MoveChuckContact": '37',
    # Moves the chuck Z axis to the contact height. In Search mode, it searches for contact height using edge sensor.
    # Parameters:
    # - Mode (optional): 'N' for normal mode, 'S' for search mode (default: N).
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckLoad": '3A',  
    # Moves the chuck stage to the load position. Can optionally turn off the vacuum.
    # Parameters:
    # - TurnOffVacuum (optional byte): 1 to turn off vacuum, 0 to keep vacuum on (default is 0).

    "MoveChuckPosition": '3C',
    # General-purpose move command for the chuck stage, does not use Z mode.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckVelocity": '3D',
    # - Moves the chuck stage in velocity mode. The motion continues until the StopChuckMovement command
    # is received, or the end limit (error condition) is reached.
    # - xValue: activate axis
    # - yValue: activate axis
    # - zValue: activate axis
    # - Velocity : %Velocity 

    "MoveChuckSeperation": '39', 
    # Moves the chuck Z axis to the separation height.
    # Parameters: None

    "MoveChuckZ": '3B',  
    # Moves the chuck Z axis to a specified height.
    # Parameters:
    # - zValueFloat: Target Z height.
    # - PosRefChar (optional): Reference position. 'Z' for abs from zero (default), 'H' for abs from contact, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "ReadChuckHeights": '32',  
    # Returns the current settings for chuck Z movement (contact height, overtravel, align, separation, load).
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.

    "ReadChuckPosition": '31', 
    # Returns the current position of the chuck stage in X, Y, and Z.
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - PosRef (optional): Position reference. 'H' for from home (default), 'Z' for from zero, 'C' for from center.
    # - CompMode (optional): Compensation mode. 'M' for matrix comp, 'L' for linear comp only, 'N' for no comp.

    "ReadChuckStatus": '30',
    # Returns the current status of the chuck.
    # Parameters: None

    "SetChuckHeight": '42',
    # Defines predefined contact height and gaps for overtravel, align, load, and separation.
    # Parameters:
    # - Level: Contact height, overtravel, align, or separation height.
    # - Mode (optional): '0' for current position, 'R' for reset contact height, 'V' for take value.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Value (optional): Height value in specified units.

    "SetChuckHome": '40',
    # Sets the current chuck position as the wafer home position.
    # Parameters:
    # - Mode (optional): '0' for current position (default), 'V' for take value.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - xValueFloat: X position.
    # - yValueFloat: Y position.

    "SetChuckMode": '3F',
    # Sets the chuck contact control mode.
    # Parameters:
    # - Overtravel (byte): 0=Off, 1=On, 2=Ignore.
    # - Auto Z (byte): 0=Off, 1=On, 2=Ignore.
    # - Interlock (byte): 0=Off, 1=On, 2=Ignore.
    # - Edge Sensor (byte): 0=Off, 1=On, 2=Ignore.
    # - Platen Stroke (byte): Reserved.

    "StopChuckMovement": '3E',
    # Stops all chuck movements in X, Y, and Z.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is all axes).

    # Scope Commands
    "InitScope": '73',  
    # Initializes the microscope stage in X, Y, and Z. Default is XY in minus and Z in plus.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is all axes).
    # - Direction (optional byte): Bit 00=X-, 1=X+, Bit 10=Y-, 1=Y+, Bit 20=Z-, 1=Z+.

    "MoveLiftDown": '77',
    # Moves the microscope lift to the lower position.
    # Parameters: None

    "MoveLiftUp": '76',
    # Moves the microscope lift to the upper position.
    # Parameters: None

    "MoveScope": '74',
    # Moves the microscope stage to the specified X, Y position relative to the home position.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopeIndex": '75',
    # Moves the microscope stage in index steps.
    # Parameters:
    # - xValueInt: X index steps.
    # - yValueInt: Y index steps.
    # - PositionRefChar (optional): Position reference. 'H' for abs from home (default), 'R' for rel from current.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopePosition": '79',
    # General-purpose move command for the scope stage.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopeVelocity": '7A',
    # Moves the microscope stage in velocity mode.
    # Parameters:
    # - xAxisChar: Direction for X axis ('+' or '-' or '0').
    # - yAxisChar: Direction for Y axis ('+' or '-' or '0').
    # - zAxisChar: Direction for Z axis ('+' or '-' or '0').
    # - VelocityFloat: Movement velocity in percentage (max 100.0).

    "MoveScopeZ": '78',
    # Moves the microscope Z axis to a specified height.
    # Parameters:
    # - zValueFloat: Target Z height.
    # - PosRefChar (optional): Reference position. 'Z' for abs from zero (default), 'H' for abs from contact, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "ReadScopePosition": '71',
    # Returns the current position of the microscope stage in X, Y, and Z.
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - PosRef (optional): Position reference. 'H' for from home (default), 'Z' for from zero, 'C' for from center.
    # - CompMode (optional): Compensation mode. 'M' for matrix comp, 'L' for linear comp only, 'N' for no comp.

    "ReadScopeStatus": '70',
    # Returns the current status of the microscope.
    # Parameters: A lot
}


# Example usage: sending a command to the probe bench
# def send_command(command_name):
#     if command_name in command_dict:
#         ascii_command = command_dict[command_name]
#         probe_bench.write_raw(ascii_command)  # Send the ASCII command directly
#     else:
#         print(f"Command {command_name} not found in the dictionary.")

# # Example: sending the "CHUCK_HOME" command
# send_command("CHUCK_HOME")


**PROBER MOVEMENT GUI**

In [3]:
import tkinter as tk

class ChuckControlApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Chuck Control")

        # Initialize position variables
        self.x_pos = 0
        self.y_pos = 0
        self.z_pos = 0

        # Create GUI elements
        self.create_widgets()

    def create_widgets(self):
        # Create D-Pad
        self.dpad_frame = tk.Frame(self.root)
        self.dpad_frame.grid(row=0, column=0, padx=10, pady=10)

        self.btn_up = tk.Button(self.dpad_frame, text="Up", command=self.move_up)
        self.btn_up.grid(row=0, column=1)

        self.btn_left = tk.Button(self.dpad_frame, text="Left", command=self.move_left)
        self.btn_left.grid(row=1, column=0)

        self.btn_right = tk.Button(self.dpad_frame, text="Right", command=self.move_right)
        self.btn_right.grid(row=1, column=2)

        self.btn_down = tk.Button(self.dpad_frame, text="Down", command=self.move_down)
        self.btn_down.grid(row=2, column=1)

        self.btn_home = tk.Button(self.dpad_frame, text="Home", command=self.move_home)
        self.btn_home.grid(row=1, column=1)

        # Create Z-axis control buttons
        self.btn_z_up = tk.Button(self.root, text="Move Z Up", command=self.move_z_up)
        self.btn_z_up.grid(row=1, column=1, padx=10, pady=10)

        self.btn_z_down = tk.Button(self.root, text="Move Z Down", command=self.move_z_down)
        self.btn_z_down.grid(row=2, column=1, padx=10, pady=10)
        
    def move_up(self):
        self.y_pos += 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_down(self):
        self.y_pos -= 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_left(self):
        self.x_pos -= 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_right(self):
        self.x_pos += 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_home(self):
        self.x_pos = 0
        self.y_pos = 0
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_z_up(self):
        self.z_pos += 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def move_z_down(self):
        self.z_pos -= 1
        self.send_command('MOVE', self.x_pos, self.y_pos, self.z_pos)

    def send_command(self, command, x, y, z):
        # Print the command to verify output
        print(f"Sending command: {command} X:{x} Y:{y} Z:{z}")
        # Here you should add code to send the command to the chuck.
        # Ensure this is non-blocking or done asynchronously if it involves I/O operations.

if __name__ == "__main__":
    root = tk.Tk()
    app = ChuckControlApp(root)
    root.mainloop()


**Command Sending**

In [24]:
def SendCommand(instrument, commandName, **kwargs):
    # Access the parameters using the kwargs dictionary
    xValue = kwargs.get('X', None)
    yValue = kwargs.get('Y', None)
    zValue = kwargs.get('Z', None)
    PosRefChar = kwargs.get('RefChar', None)
    Unit = kwargs.get('Unit', None)
    V = kwargs.get('V', None)
    Mode = kwargs.get('Mode', None)
    Velocity = kwargs.get('Velocity', None)
    CompMode = kwargs.get('CompMode', None)
    Level = kwargs.get('Level', None)
    Overtravel = kwargs.get('Overtravel', None)
    AutoZ = kwargs.get('AutoZ', None)
    Interlock = kwargs.get('Interlock', None)
    EdgeSensor = kwargs.get('EdgeSensor', None)
    Axis = kwargs.get('Axis', None)
    Direction = kwargs.get('Direction', None)
    
    if commandName in command_dict:
        # Retrieve the base command template
        asciiCommand = command_dict[commandName]
        
        # Initialize an empty list to hold command parts
        command_parts = [asciiCommand]
        
        # Append parameters dynamically based on kwargs
        for key, value in kwargs.items():
            if value is not None:
                # Format each parameter (assuming space-separated format)
                command_parts.append(f'{value}')
        
        # Join all parts into a single command string
        full_command = ' '.join(command_parts)
        
        # Send the complete command
        print(full_command)
#         instrument.write(full_command)
    else:
        print(f"Command {commandName} not found in command_dict")
    
    return

def ChuckSetup():
    SendCommand(probe_bench, 'InitChuck')
    SendCommand(probe_bench, 'SetChuckMode', EdgeSensor = 1)
    SendCommand(probe_bench, 'MoveChuckVelocity', xValue = '-', yValue = '-', zValue = '0', Velocity = 30) #change Z to also move but this is set to 0 to test so I dont crash the stage
    SendCommand(probe_bench, 'SetChuckHome')
    
    
    return

def ScopeSetup(): #needs work fix the ReadScopeStatus
    SendCommand(probe_bench, 'InitScope')
    SendCommand(probe_bench, 'SetScopeMode')
    return
probe_bench = 1
# SendCommand(instrument = probe_bench, commandName = 'InitChuck', xValue = 10, yValue = 10, zValue = 30, Unit = 'Y')
ChuckSetup()

33
3F 1
3D - - 0 30
40
